In [6]:
import pickle
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy

In [7]:
current_directory = os.getcwd()
print("Current working directory:", current_directory)

Current working directory: /Users/blaubach/chimes_CGD-myLLFork/dissimilarity_clustering


In [8]:
# Get the current working directory
cwd = os.getcwd()

# Define file paths with cwd appended
file_path_2b = os.path.join(cwd, "dft_pds/2b_all_pd")
file_path_3b = os.path.join(cwd, "dft_pds/3b_all_pd")
file_path_4b = os.path.join(cwd, "dft_pds/4b_all_pd")
file_path_labels = os.path.join(cwd, "dft_pds/labels_pd")
file_path_natoms = os.path.join(cwd, "test_notebooks/energies_per_atom.txt")

# Open pickle files with the updated file paths
with open(file_path_2b, 'rb') as pickle_file:
    pd_2b = pickle.load(pickle_file)

with open(file_path_3b, 'rb') as pickle_file:
    pd_3b = pickle.load(pickle_file)

with open(file_path_4b, 'rb') as pickle_file:
    pd_4b = pickle.load(pickle_file)

with open(file_path_labels, 'rb') as pickle_file:
    labels = pickle.load(pickle_file)

natom_list = []

# Open the text file for reading
with open(file_path_natoms, 'r') as file:

    # Read the contents of the file
    lines = file.readlines()[1:]

    # Iterate through each line
    for line in lines:

        # Split the line into words
        words = line.split()
        natoms = line.split("|")[1].strip()

        # Extract the last word, assuming it's a number
        last_number = float(words[-1])
        if words[0][-1] == 'z':
            print(words[0])
            continue
        natom_list.append(natoms)

natom_list = np.array(natom_list[:-10])

# Combine the arrays along the second axis (axis=1)
all_array = np.concatenate((pd_2b, pd_3b, pd_4b), axis=2)
all_array = all_array.reshape(-1, all_array.shape[2])
print(np.shape(all_array))
df_fingerprints = pd.DataFrame(all_array)

# # Define the column labels for each set of columns
# column_labels_2b = [f'2B_{i}' for i in range(60)]
# column_labels_3b = [f'3B_{i}' for i in range(60)]
# column_labels_4b = [f'4B_{i}' for i in range(60)]

# # Assign the column labels to the DataFrame
# column_labels = column_labels_2b + column_labels_3b + column_labels_4b
# df_fingerprints.columns = column_labels

# # Add a new column "labels" to the DataFrame and assign the new vector to it
# df_fingerprints['labels'] = labels

# # Calculate the row-wise mean using `mean(axis=1)`
# row_avg = df_fingerprints.mean(axis=1)

# # Append the calculated row-wise mean as a new column named "Pavg"
# df_fingerprints['Pavg'] = row_avg

# # Calculate the row-wise standard deviation using `std(axis=1)`
# row_std = df_fingerprints.std(axis=1)

# # Append the calculated row-wise standard deviation as a new column named "Pstd"
# df_fingerprints['Pstd'] = row_std

# # Append Natoms to Dataframe
# df_fingerprints['Natoms'] = natom_list



training_data_#0000.xyz
training_data_#0050.xyz
training_data_#0055.xyz
training_data_#0060.xyz
training_data_#0075.xyz
training_data_#0080.xyz
training_data_#0110.xyz
(300, 180)


In [9]:
# from scipy.spatial.distance import cdist

# def update_configuration_sets(min_pij_index, training_set, configuration_set, pij_matrix):
#     training_set = pd.concat([training_set, pd.DataFrame([configuration_set.loc[min_pij_index]])])
#     pij_matrix = pij_matrix.drop(min_pij_index)
#     configuration_set.drop(min_pij_index, inplace=True)
#     return training_set, configuration_set, pij_matrix

# training_set = pd.DataFrame
# for i, row in df_fingerprints.iterrows():
#     # Obtain fingerprint
#     fingerprint = row.iloc[:-4]

#     # Compute distances to all clusters in the original data
#     distances_to_clusters = cdist([fingerprint], available_cgs) # Computes the 2-norm between point and ground truth data

#     # Find the index of the closest cluster in the original data
#     farthest_index = np.argmin(distances_to_clusters)

In [10]:
from scipy.spatial.distance import cdist

def construct_comparison(training_set, candidates, pij_matrix=None):
    column_values = training_set.index.tolist()
    if pij_matrix is None:
        pij_matrix = pd.DataFrame(columns=column_values)

    pij_values = []
    phi1 = training_set.iloc[-1]

    # Ensure phi1 contains only numeric data
    phi1_numeric = pd.to_numeric(phi1, errors='coerce')
    if phi1_numeric.isnull().any():
        raise ValueError("phi1 contains non-numeric data")

    # Ensure candidates contain only numeric data
    candidates.fillna(0, inplace=True)
    for col in candidates.columns:
        candidates_numeric = pd.to_numeric(candidates[col], errors='coerce')
        if candidates_numeric.isnull().any():
            raise ValueError(f"Candidates column '{col}' contains non-numeric data")

    # Compute pairwise distances
    pij_values.append(cdist([phi1_numeric], candidates_numeric))
    pij_matrix[column_values[-1]] = pij_values

    # Calculate the average of values in each row
    means = pij_matrix.mean(axis=1)

    min_pij_index = means.idxmin()
    # Update training set, configurations, and pij_matrix
    update_training, update_configurations, update_pij_matrix = update_configuration_sets(min_pij_index, training_set, candidates, pij_matrix)
    return update_training, update_configurations, update_pij_matrix


def update_configuration_sets(min_pij_index, training_set, configuration_set, pij_matrix):
    training_set = pd.concat([training_set, pd.DataFrame([configuration_set.loc[min_pij_index]])])
    pij_matrix = pij_matrix.drop(min_pij_index)
    configuration_set.drop(min_pij_index, inplace=True)
    #configuration_set.reset_index(drop=True, inplace=True)  # Resetting indices without reindexing
    return training_set, configuration_set, pij_matrix

print(np.shape(df_fingerprints))

# Choose arbitrary starting point
starting_frame = 0
phi1 = df_fingerprints.iloc[starting_frame]
pij_matrix = None

# Initialize training set
current_training_set = pd.DataFrame(columns=df_fingerprints.columns)
current_training_set = pd.concat([current_training_set, pd.DataFrame([phi1])])

available_configurations = copy.copy(df_fingerprints)
available_configurations.drop(starting_frame, inplace=True)

# Compute pij matrix
cfg_cnt = 50
for i in range(cfg_cnt-1):
    current_training_set, available_configurations, pij_matrix = construct_comparison(current_training_set, available_configurations, pij_matrix)
plt.plot(current_training_set.index)
# Add a horizontal red line at y=24
plt.axhline(y=24, color='red')

# Show the plot
plt.show()


(300, 180)


ValueError: XB must be a 2-dimensional array.